In [1]:
import sys
import os
file_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Change this if your file is in a different folder
os.chdir(file_dir)

print(file_dir)

/home/dung/Code/Project/intro-ds-project


In [2]:
import pandas as pd
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

from utils.mongodb import load_collection_to_dataframe

In [3]:
df_BTC = load_collection_to_dataframe('raw_BTC')
df_BTC


,datetime,coin,high,volumeto,conversionType,low,open,close,time,volumefrom,conversionSymbol
0,2020-01-01 00:00:00,BTC,7198.83,5029316.72,direct,7174.38,7183.88,7174.39,1577836800,699.30,
1,2020-01-01 01:00:00,BTC,7230.63,6209785.62,direct,7171.82,7174.39,7202.75,1577840400,861.69,
2,2020-01-01 02:00:00,BTC,7248.41,4581067.47,direct,7202.75,7202.75,7237.29,1577844000,633.44,
3,2020-01-01 03:00:00,BTC,7242.14,4297949.00,direct,7209.84,7237.29,7210.09,1577847600,593.02,
4,2020-01-01 04:00:00,BTC,7235.42,3406138.58,direct,7207.97,7210.09,7209.83,1577851200,469.47,
...,...,...,...,...,...,...,...,...,...,...,...
46407,2025-04-23 03:00:00,BTC,93235.24,68746230.62,direct,92724.58,92898.58,93190.14,1745377200,739.88,
46408,2025-04-23 04:00:00,BTC,93706.93,52829567.91,direct,93059.03,93190.14,93694.59,1745380800,566.26,
46409,2025-04-23 05:00:00,BTC,93694.59,59313196.82,direct,93327.39,93694.59,93511.20,1745384400,634.46,
46410,2025-04-23 06:00:00,BTC,93891.42,41927067.45,direct,93454.86,93511.20,93664.38,1745388000,447.47,


In [4]:
import pandas as pd

def clean_raw_data(df):
    """
    This function removes unnecessary columns from the raw data and renames columns for clarity.

    Parameters:
    df (pandas.DataFrame): The raw DataFrame with OHLCV data.

    Returns:
    pandas.DataFrame: The cleaned DataFrame with unnecessary columns removed and renamed columns.
    """
    # Remove unnecessary columns
    df_cleaned = df.drop(columns=['coin', 'conversionType', 'conversionSymbol'])
    
    # Rename columns for clarity
    df_cleaned = df_cleaned.rename(columns={
        'high': 'high_price',
        'low': 'low_price',
        'open': 'open_price',
        'close': 'close_price',
        'volumeto': 'volume_to',
        'volumefrom': 'volume_from'
    })
    
    return df_cleaned

In [5]:
df_BTC = clean_raw_data(df_BTC)

In [6]:
import ta


def calculate_technical_indicators(df):
    # 1. Calculate On-Balance Volume (OBV)
    def calculate_obv(df):
        """Manually calculate On-Balance Volume (OBV)"""
        obv = [0]
        for i in range(1, len(df)):
            if df['close_price'][i] > df['close_price'][i - 1]:  # price increase
                obv.append(obv[-1] + df['volume_to'][i])
            elif df['close_price'][i] < df['close_price'][i - 1]:  # price decrease
                obv.append(obv[-1] - df['volume_to'][i])
            else:  # no change
                obv.append(obv[-1])
        return obv

    df['obv'] = calculate_obv(df)

    # 2. Calculate Accumulation/Distribution Line (A/D)
    def calculate_ad_line(df):
        """Manually calculate the Accumulation/Distribution (A/D) line"""
        ad_line = [0]  # Start AD at 0
        for i in range(1, len(df)):
            mf_multiplier = ((df['close_price'][i] - df['low_price'][i]) - (df['high_price'][i] - df['close_price'][i])) / (df['high_price'][i] - df['low_price'][i])
            mf_volume = mf_multiplier * df['volume_to'][i]
            ad_line.append(ad_line[-1] + mf_volume)
        return ad_line

    df['acc_dist'] = calculate_ad_line(df)

    # 3. Calculate Average Directional Index (ADX)
    df['adx'] = ta.trend.ADXIndicator(df['high_price'], df['low_price'], df['close_price'], window=14).adx()

    # 4. Calculate Aroon Indicator (Aroon Up, Aroon Down)
    df['aroon_up'] = ta.trend.AroonIndicator(df['high_price'], df['low_price'], window=14).aroon_up()
    df['aroon_down'] = ta.trend.AroonIndicator(df['high_price'], df['low_price'], window=14).aroon_down()

    # 5. Calculate MACD (Moving Average Convergence Divergence)
    df['macd'] = ta.trend.MACD(df['close_price']).macd()

    # 6. Calculate Relative Strength Index (RSI)
    df['rsi'] = ta.momentum.RSIIndicator(df['close_price'], window=14).rsi()

    # 7. Calculate Stochastic Oscillator
    df['stoch'] = ta.momentum.StochasticOscillator(df['high_price'], df['low_price'], df['close_price'], window=14).stoch()

    return df

# Example usage:
# Assuming you have a DataFrame `df` with OHLCV data:
# df = pd.read_csv('your_data.csv')  # Example for reading CSV data


In [7]:
df_BTC = calculate_technical_indicators(df_BTC)
df_BTC

,datetime,high_price,volume_to,low_price,open_price,close_price,time,volume_from,obv,acc_dist,adx,aroon_up,aroon_down,macd,rsi,stoch
0,2020-01-01 00:00:00,7198.83,5029316.72,7174.38,7183.88,7174.39,1577836800,699.30,0.000000e+00,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 01:00:00,7230.63,6209785.62,7171.82,7174.39,7202.75,1577840400,861.69,6.209786e+06,3.220515e+05,0.000000,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 02:00:00,7248.41,4581067.47,7202.75,7202.75,7237.29,1577844000,633.44,1.079085e+07,2.671780e+06,0.000000,NaN,NaN,NaN,NaN,NaN
3,2020-01-01 03:00:00,7242.14,4297949.00,7209.84,7237.29,7210.09,1577847600,593.02,6.492904e+06,-1.559637e+06,0.000000,NaN,NaN,NaN,NaN,NaN
4,2020-01-01 04:00:00,7235.42,3406138.58,7207.97,7210.09,7209.83,1577851200,469.47,3.086766e+06,-4.504179e+06,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46407,2025-04-23 03:00:00,93235.24,68746230.62,92724.58,92898.58,93190.14,1745377200,739.88,2.635622e+10,1.195981e+11,59.531718,57.142857,0.0,1328.088090,73.102815,80.601488
46408,2025-04-23 04:00:00,93706.93,52829567.91,93059.03,93190.14,93694.59,1745380800,566.26,2.640905e+10,1.196489e+11,60.760950,50.000000,0.0,1349.194858,75.484061,92.985339
46409,2025-04-23 05:00:00,93694.59,59313196.82,93327.39,93694.59,93511.20,1745384400,634.46,2.634973e+10,1.196490e+11,61.902381,42.857143,0.0,1335.726657,72.955363,86.768676
46410,2025-04-23 06:00:00,93891.42,41927067.45,93454.86,93511.20,93664.38,1745388000,447.47,2.639166e+10,1.196473e+11,63.042176,35.714286,0.0,1322.172179,73.746479,91.462510


In [8]:
# Drop rows where any column has NaN (due to rolling calculation)
df_BTC = df_BTC.dropna().reset_index(drop=True)
df_BTC


,datetime,high_price,volume_to,low_price,open_price,close_price,time,volume_from,obv,acc_dist,adx,aroon_up,aroon_down,macd,rsi,stoch
0,2020-01-02 01:00:00,7224.84,3525833.70,7186.18,7218.61,7195.19,1577926800,487.16,-1.093437e+07,-3.805598e+06,0.000000,50.000000,85.714286,4.270893,43.967544,20.483924
1,2020-01-02 02:00:00,7217.72,7263407.16,7153.37,7195.19,7180.68,1577930400,1012.45,-1.819778e+07,-4.903857e+06,0.000000,42.857143,100.000000,0.793421,40.480699,24.000352
2,2020-01-02 03:00:00,7185.49,10320976.65,7127.64,7180.68,7130.25,1577934000,1444.77,-2.851876e+07,-1.429354e+07,32.584263,35.714286,100.000000,-5.963042,31.215136,1.870700
3,2020-01-02 04:00:00,7172.20,5756869.86,7124.34,7130.25,7154.73,1577937600,803.86,-2.276189e+07,-1.273945e+07,33.099357,28.571429,100.000000,-9.235791,38.566021,21.278532
4,2020-01-02 05:00:00,7163.85,3073480.43,7128.92,7154.73,7138.33,1577941200,428.78,-2.583537e+07,-1.415696e+07,33.577658,21.428571,92.857143,-13.002922,35.805367,9.795547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46382,2025-04-23 03:00:00,93235.24,68746230.62,92724.58,92898.58,93190.14,1745377200,739.88,2.635622e+10,1.195981e+11,59.531718,57.142857,0.000000,1328.088090,73.102815,80.601488
46383,2025-04-23 04:00:00,93706.93,52829567.91,93059.03,93190.14,93694.59,1745380800,566.26,2.640905e+10,1.196489e+11,60.760950,50.000000,0.000000,1349.194858,75.484061,92.985339
46384,2025-04-23 05:00:00,93694.59,59313196.82,93327.39,93694.59,93511.20,1745384400,634.46,2.634973e+10,1.196490e+11,61.902381,42.857143,0.000000,1335.726657,72.955363,86.768676
46385,2025-04-23 06:00:00,93891.42,41927067.45,93454.86,93511.20,93664.38,1745388000,447.47,2.639166e+10,1.196473e+11,63.042176,35.714286,0.000000,1322.172179,73.746479,91.462510


In [9]:
from utils.mongodb import save_dataframe_to_collection

save_dataframe_to_collection(df_BTC, 'BTC_technical_indicators')

In [10]:
# import pandas as pd

# def select_top_features(df, target_column, top_n=9):
#     # Calculate the correlation matrix
#     correlation_matrix = df.corr()

#     # Get the absolute correlation values for the target variable
#     target_corr = correlation_matrix[target_column].abs()

#     # Sort the features by correlation in descending order and select the top_n
#     top_features = target_corr.sort_values(ascending=False).head(top_n+1).index.tolist()

#     # Drop the target column itself from the list (since it's not a feature)
#     top_features.remove(target_column)

#     # Return the DataFrame with the top N features
#     return df[top_features + [target_column]]

# # Example usage:
# # Assuming 'target' is your target column, replace 'target' with your actual target column name
# df_top_features = select_top_features(df_BTC, target_column='close_price', top_n=9)
# print(df_top_features.head())


In [11]:
# raw_btc = db["raw_BTC"]
# raw_btc_df = pd.DataFrame(list(raw_btc.find({})))

In [12]:
# print(raw_btc_df.info())